In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchlensmaker as tlm


class Optics(tlm.Module):
    def __init__(self):
        super().__init__()
        self.shape = tlm.Parabola(width=15., a=nn.Parameter(torch.tensor(0.005)))

        self.lens = tlm.SymmetricLens(self.shape, (1.0, 1.49), outer_thickness=0.5)

        self.optics = nn.Sequential(
            tlm.ParallelBeamUniform(width=15.),
            tlm.GapY(10.),
            
            self.lens,
            
            tlm.GapY(45.0),
            tlm.FocalPointLoss(),
        )

    def forward(self, inputs):
        return self.optics(inputs)

optics = Optics()

tlm.render_plt(optics, (10, torch.tensor([0., 0.])))

tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    inputs = (10, torch.tensor([0., 0.])),
    num_iter = 100
)

tlm.render_plt(optics, (10, torch.tensor([0., 0.])))

In [ ]:
from IPython.display import display
import build123d as bd

part = tlm.lens_to_part(optics.lens)
display(part)

print("Outer thickness:", optics.lens.outer_thickness().item())
print("Inner thickness:", optics.lens.inner_thickness().item())

bd.export_step(part, "lens.step")